# Section1: Experiment Search Console


In [1]:
import os,sys,inspect
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../../"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../components/"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../dashboards/"))

import SystemConsole
import  BokehComponents
from BokehComponents import *


Loading BokehJS ...

In [2]:
class ProcessQuery(BufferedQueryInterface):
    def load_data_buffer(self):
        if hasattr(self, 'data'):
            pass
        else:
            self.data = {
                'process_id':[str(i) for i in range(100)],
                'file':['process_name' for i in range(100)],
                'memory':[ i%10+10 for i in range(100)],
                'cpu':[ i%5+10 for i in range(100)],
                }
            self.actions = {'kill':self.action_kill}
    
    def action_kill(self,ids):
        for k in self.data.keys(): 
            for selected_index in reversed(ids):
                self.data[k].pop(selected_index)

                
                
class ExperimentQuery(BufferedQueryInterface):
    def load_data_buffer(self):
        if hasattr(self, 'data'):
            pass
            for i in range(0,len(self.data['duration'])):
                self.data['duration'][i] = self.data['duration'][i]+1 
        else:
            self.data = {
                'eid':['eid_'+str(i) for i in range(4)],
                'job':['experiment ' for i in range(4)],
                'status':['running ' for i in range(4)],
                'duration':[0 for i in range(4)],
                }
            self.actions = {'kill':self.action_kill}
    
    def action_kill(self,ids):
        for k in self.data.keys(): 
            for selected_index in reversed(ids):
                self.data[k].pop(selected_index)

                
class AlgorithmQuery(BufferedQueryInterface):
    def load_data_buffer(self):
        if hasattr(self, 'data'):
            pass
            for i in range(0,len(self.data['duration'])):
                self.data['duration'][i] = self.data['duration'][i]+1 
        else:
            self.data = {
                'aid':['aid_'+str(i) for i in range(10)],
                'job':['algorithm ' for i in range(10)],
                'status':['running ' for i in range(10)],
                'duration':[0 for i in range(10)],
                }
            self.actions = {'kill':self.action_kill}
    
    def action_kill(self,ids):
        for k in self.data.keys(): 
            for selected_index in reversed(ids):
                self.data[k].pop(selected_index)                
#------- usage    
        
#pi = ProcessQuery()
#df = pd.DataFrame(pi.QueryData())
#df = pd.DataFrame(pi.QueryData(query={'key':'process_id','value':'6'}))
#display(df)
#pi.DoAction(action_id = 'kill', query={'key':'file','value':'jobStatrer.pyc '} )
#pi.DoAction(action_id = 'kill', query={'key':'process_id','value':'6'} )
#df = pd.DataFrame(pi.QueryData())
#display(df)


In [3]:
# Define our buttions
class KillButton(BokehButton):
    def handle_click(self,event):
        for t in self._settings['tables_to_kill']:
            t.removeSelected()


class ShowButton(BokehButton):
    def handle_click(self,event):
        rows = []
        for targetTable in self._settings['analytics_targets']:
            selected = targetTable.getSelected()
            if len(selected) > 0:
                selected = selected[0]
                row = {}
                for col in targetTable.data.keys(): 
                    row[col]=targetTable.data[col][selected]
                rows.append(row)
        ## Load the experiment manager, and get analytics
        
        ad = self._settings['analytics_div']
        ad.text = str(rows)

In [4]:
from datetime import date,datetime,timedelta
from random import randint
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox, Spacer
from bokeh.models import ColumnDataSource,CustomJS,Div
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Tabs, Panel
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import layout
from bokeh.events import ButtonClick
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
import pandas as pd
import pandas_datareader as pdr
from bokeh.plotting import figure,show,output_file

from BokehComponents import *
from SystemConsole import *

output_notebook()
output_file("data_table.html")
class FinancialConsole:
    def __init__(self):
        self.div = Div(width=300)
        self.pt  = QueryTableComponent({'buffered_query_interface':ProcessQuery(),'id_field':'process_id'})        
        self.et  = QueryTableComponent({'buffered_query_interface':ExperimentQuery(),'id_field':'eid'})        
        self.at  = QueryTableComponent({'buffered_query_interface':AlgorithmQuery(),'id_field':'aid'})
        
        self.analytics_div = Div(text="""Analytics HTML""", width=200, height=100)
        self.console = Div(text="""System Out console:""", width=200, height=100)
        
        self.b_kill = KillButton({'tables_to_kill':[self.pt,self.et,self.at],'label':'kill'})
        self.b_show = ShowButton({'label':'show','analytics_targets':[self.et,self.at],'analytics_div':self.analytics_div})
        self.b_restart = ShowButton({'label':'restart'})
        #self.time_graphic = TimeseriesGraphic()
        #self.periodicCallbacks = [self.time_graphic.callback_refresh_analytics]
        self.periodicCallbacks = []
        
        self.s_adviceSelect = BokehSelect({'title':"Advice On:", 'value':"Auto", 'options':["Yes", "No", "Auto"]})
        self.s_tradeSelect = BokehSelect({'title':"Trade:", 'value':"Auto", 'options':["Yes", "No", "Auto"]})
        self.s_autoRestart = BokehSelect({'title':"Autostart:", 'value':"No", 'options':["Yes", "No"]})
        
    def getPeriodicCallbacks(self):
        return self.periodicCallbacks
        
    def getControlPanel(self):
        
        bk = self.b_kill.getBokehComponent()
        br = self.b_restart.getBokehComponent()
        bs = self.b_show.getBokehComponent()
        adviceSelect = self.s_adviceSelect.getBokehComponent()
        tradeSelect  = self.s_tradeSelect.getBokehComponent()
        autoRestart  = self.s_autoRestart.getBokehComponent()
        
        c_tab1 = Panel(child=layout([bk,bs,br], sizing_mode='fixed'),title="Commands")
        c_tab2 = Panel(child=layout([adviceSelect,tradeSelect,autoRestart], sizing_mode='fixed'),title="Signals")
        c_tabs = Tabs(tabs=[ c_tab1,c_tab2], width=300)
        
        return widgetbox(c_tabs)
    


    def getConsole(self):

        #(Compute Nodes, Running Experiments, Queue and History, Orders & Advice )
        #financePlot = self.time_graphic.getBokehComponent()

        j_tab1 = Panel(child=layout([self.analytics_div], sizing_mode='fixed'),title="Analytics")
        j_tab2 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="System Out")
        j_tab3 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="Log Data")
        j_tab4 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="Job Code")
        j_tabs = Tabs(tabs=[ j_tab1, j_tab2, j_tab3, j_tab4 ], width=700)

        
        l1 = layout([
          [
            [self.pt.getBokehComponent()]         
          ],
           [self.getControlPanel()],
            [self.console]            
        ],sizing_mode='fixed')         
        
        l2 = layout([
          [
            [self.et.getBokehComponent()],
                Spacer(width=20),
           [self.at.getBokehComponent()]          
          ],
           [self.getControlPanel(),j_tabs],
        ],sizing_mode='fixed') 
        

        tab1 = Panel(child=l1,title="System Processes Test")
        tab2 = Panel(child=l2,title="Running Experiments")
        tab3 = Panel(child=Div(height=300,width=600),title="Orders & Integration")
        tabs = Tabs(tabs=[ tab1, tab2, tab3 ])
        return tabs
    
SystemConsole.FinancialConsole = FinancialConsole

Loading BokehJS ...

In [5]:
SystemConsole.showConsole("ec2.paxculturastudios.com:9999")

Show EVENT
[2]
build------------------------
targetTable.data {'eid': ['eid_0', 'eid_1', 'eid_2', 'eid_3'], 'job': ['experiment ', 'experiment ', 'experiment ', 'experiment '], 'status': ['running ', 'running ', 'running ', 'running '], 'duration': [2, 2, 2, 2]}
targetTable.data[col] ['eid_0', 'eid_1', 'eid_2', 'eid_3']
targetTable.data[selected] eid_2
endbuild------------------------
build------------------------
targetTable.data {'eid': ['eid_0', 'eid_1', 'eid_2', 'eid_3'], 'job': ['experiment ', 'experiment ', 'experiment ', 'experiment '], 'status': ['running ', 'running ', 'running ', 'running '], 'duration': [2, 2, 2, 2]}
targetTable.data[col] ['experiment ', 'experiment ', 'experiment ', 'experiment ']
targetTable.data[selected] experiment 
endbuild------------------------
build------------------------
targetTable.data {'eid': ['eid_0', 'eid_1', 'eid_2', 'eid_3'], 'job': ['experiment ', 'experiment ', 'experiment ', 'experiment '], 'status': ['running ', 'running ', 'running ', 

Show EVENT
[1]
build------------------------
targetTable.data {'eid': ['eid_0', 'eid_3'], 'job': ['experiment ', 'experiment '], 'status': ['running ', 'running '], 'duration': [10, 10]}
targetTable.data[col] ['eid_0', 'eid_3']
targetTable.data[selected] eid_3
endbuild------------------------
build------------------------
targetTable.data {'eid': ['eid_0', 'eid_3'], 'job': ['experiment ', 'experiment '], 'status': ['running ', 'running '], 'duration': [10, 10]}
targetTable.data[col] ['experiment ', 'experiment ']
targetTable.data[selected] experiment 
endbuild------------------------
build------------------------
targetTable.data {'eid': ['eid_0', 'eid_3'], 'job': ['experiment ', 'experiment '], 'status': ['running ', 'running '], 'duration': [10, 10]}
targetTable.data[col] ['running ', 'running ']
targetTable.data[selected] running 
endbuild------------------------
build------------------------
targetTable.data {'eid': ['eid_0', 'eid_3'], 'job': ['experiment ', 'experiment '], 'status

In [7]:
### Job Management
# [ ] select: Queue, Running, Halted, Finihsed
# [ ] search: MongoDB settings query 
# [ ] Parent / Child Filtering
# [ ] Parent = All Jobs present
# [ ] Parent = No Parent
# [X] running experiments in tab 2
# [X] get kills working
# [X] get show button working
# [ ] get sub experiments listing based on experiment
# [ ] autostart property on Experiment
# [ ] See job settings

### Orders & Advice System
# [ ] See linked in traction
# [ ] For each Experiment (running / historical):
# [ ] Sent Brackets + P/L
# [ ] Pending Brackets + P/L
# [ ] Raw P/L
# [ ] Advice (On / Off)
# [ ] Trading (On / Off)



'TimeseriesGraphic' object has no attribute 'ds1'


/home/ec2-user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 67), ('y', 65)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/home/ec2-user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 67), ('y', 65)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1): KeyError('pid',)
ERROR:bokeh.server.protocol_handler:error handling message Message 'EVENT' (revision 1): TypeError('doRemoveIndices() takes 2 positional arguments but 3 were given',)


KILL EVENT


ERROR:bokeh.server.protocol_handler:error handling message Message 'EVENT' (revision 1): TypeError('doRemoveIndices() takes 2 positional arguments but 3 were given',)


KILL EVENT


In [123]:
ee.kill()

AttributeError: 'FinancialConsole' object has no attribute 'kill'

In [4]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

# Create a blank figure with labels
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Example Glyphs',
           x_axis_label = 'X', y_axis_label = 'Y')

# Example data
squares_x = [1, 3, 4, 5, 8]
squares_y = [8, 7, 3, 1, 10]
circles_x = [9, 12, 4, 3, 15]
circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(circles_x, circles_y, size = 12, color = 'red')

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

Loading BokehJS ...

In [32]:
from pymongo import MongoClient
mf = mongoInfo
display(mf)
client=MongoClient(host=mf['mongo_server'],port=mf['port'],username=mf['username'],password=mf['password'],authSource=mf['authSource'])
display(client)

{'mongo_server': '127.0.0.1',
 'port': 27017,
 'username': 'pax_user',
 'password': 'paxuser43',
 'authSource': 'fleetRover'}

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True, authsource='fleetRover')

In [33]:
display(client.list_database_names())

['fleetRover', 'test']

In [34]:
str_db = client.list_database_names()[0]
display(str_db)

'fleetRover'

In [35]:
display(client[str_db].collection_names())


/home/ec2-user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['yahoo',
 'experiments',
 'as500',
 'experiments_data',
 'gspc',
 'message_log',
 'experiments_queue',
 'messages',
 'experiments_data_old']

In [64]:
import pprint
cur = client['fleetRover']['yahoo'].find().limit(500)
data = []
for f in cur:
    data.append(f)
df = pd.DataFrame(data)
display(df)

,Adj Close,Close,Date,High,Low,Open,Security,Volume,_id
0,52.623859,54.259998,2015-01-02,54.689999,53.790001,54.360001,C,11137800.0,5aeb4d5233b5722ad90c2e71
1,50.965424,52.549999,2015-01-05,53.959999,52.330002,53.910000,C,20945400.0,5aeb4d5233b5722ad90c2e72
2,49.171204,50.700001,2015-01-06,52.770000,50.529999,52.680000,C,31989300.0,5aeb4d5233b5722ad90c2e73
3,49.627037,51.169998,2015-01-07,51.549999,50.619999,51.509998,C,27890800.0,5aeb4d5233b5722ad90c2e74
4,50.373814,51.939999,2015-01-08,52.080002,51.500000,51.770000,C,17467200.0,5aeb4d5233b5722ad90c2e75
5,49.248791,50.779999,2015-01-09,52.240002,50.770000,52.169998,C,20164300.0,5aeb4d5233b5722ad90c2e76
6,48.860847,50.380001,2015-01-12,50.910000,50.029999,50.820000,C,22725800.0,5aeb4d5233b5722ad90c2e77
7,48.511711,50.020000,2015-01-13,51.230000,49.610001,50.660000,C,23201800.0,5aeb4d5233b5722ad90c2e78
8,47.570961,49.049999,2015-01-14,49.099998,47.770000,48.959999,C,44630800.0,5aeb4d5233b5722ad90c2e79
9,45.805840,47.230000,2015-01-15,48.240002,46.910000,48.060001,C,48799400.0,5aeb4d5233b5722ad90c2e7a


In [66]:
d = df[['High','Low']]
display(d)

,High,Low
0,54.689999,53.790001
1,53.959999,52.330002
2,52.770000,50.529999
3,51.549999,50.619999
4,52.080002,51.500000
5,52.240002,50.770000
6,50.910000,50.029999
7,51.230000,49.610001
8,49.099998,47.770000
9,48.240002,46.910000


In [67]:
# Read the data from a csv into a dataframe
# Summary stats for the column of interest
d['High'].describe()


count    500.000000
mean      50.220820
std        5.465364
min       36.160000
25%       46.089999
50%       51.205000
75%       54.269998
max       61.000000
Name: High, dtype: float64

In [72]:
#Develop a Histogram
import numpy as np
arr_hist, edges = np.histogram(d['High'], 
                               bins = int(180/5), 
                               range = [-60, 120])
# Put the information in a dataframe
delays = pd.DataFrame({'arr_delay': arr_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})

# Create the blank plot
p = figure(plot_height = 600, plot_width = 600, 
           title = 'Histogram',
          x_axis_label = 'Amount]', 
           y_axis_label = 'Number')

# Add a quad glyph
p.quad(bottom=0, top=d['Low'], 
       left=delays['left'], right=delays['right'], 
       fill_color='red', line_color='black')

# Show the plot
show(p)

/home/ec2-user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('left', 36), ('right', 36), ('top', 500)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
